# Quantum Random Number Generator - Simulation


## DV QRNG

### Core Physics 
A **qubit** is a two level quantum system with states $|0\rangle$ and $|1\rangle$.

If we prepare the qubit in the state $|0\rangle$ and apply a **Hadamard gate** $H$, the state becomes: $|+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}$

If we now measure this state in the **computational basis** $\{ |0\rangle, |1\rangle \}$, the **Born rule** says:
$P(0) = |\langle 0 | + \rangle|^2 = \tfrac{1}{2}$ and $P(1) = |\langle 1 | + \rangle|^2 = \tfrac{1}{2}$

*Intrinsic* randomness of quantum mechanics vs deterministic classical randomness (chaos). 

Even if we had complete information about the system, the measurement outcome is **fundamentally unpredictable**.  
Each measurement therefore gives one *truly random bit*.

#### **Qiskit**
- IBM’s open-source Python framework for quantum computing
- allows the simulation of quantum circuits on a classical computer
- run the same circuits on **real IBM quantum processors** in the cloud

This means that:
- on a simulator, we get *simulated quantum statistics* (generated using a classical pseudo-random number generators)  
- on a real IBM quantum device, we can obtain *genuine quantum random numbers*

$\rightarrow$ i created an IBM Quantum account and i am trying to get cloud access to real quantum chips (working on it...)


In [92]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

In [93]:
qc = QuantumCircuit(1, 1)  
qc.h(0)  
qc.measure(0, 0)

backend = AerSimulator() 
transpiled_qc = transpile(qc, backend)
job = backend.run(transpiled_qc, shots=1000000, memory=True)
result = job.result()
bitstring = np.array(result.get_memory(), dtype=np.uint8)
print('Bitstring: ', bitstring)
print('First 100 bits of Bitstring: \n', bitstring[:100])

counts = result.get_counts()
print()
print('Random bits generated: ', counts)


Bitstring:  [0 0 0 ... 1 1 1]
First 100 bits of Bitstring: 
 [0 0 0 1 0 1 1 1 0 1 1 0 0 0 1 0 0 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 1
 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 1 0]

Random bits generated:  {'1': 500058, '0': 499942}


#### **$H_{min}$ *Min-Entropy***  

Min-entropy ($H_{min}$) is a measure of the unpredictability of a random variable. It is defined as: $H_{min}(X) = -log_2(max(P(X=x)))$

- represents the worst case predictability where:
    - 0 $\rightarrow$ fully predictable
    - 1 $\rightarrow$ perfectly random (maximally unpredictable)


In [94]:
def min_entropy(counts, shots):
    probs = np.array([counts.get('0', 0)/shots, counts.get('1', 0)/shots])
    return -np.log2(np.max(probs))

shots = 1000000
h_min = min_entropy(counts, shots)
print('Min Entropy: ', h_min)

Min Entropy:  0.9998326570809586


### **Randomness Extraction Models**

Randomness Extraction Models are methods / algorithms that take biased sources of randomness and produce uniform, unbiased random bits

- Von Neumann Extractor (pair-wise)
- 

#### Von Neumann Extractor

The **von Neumann extractor** is a simple method to *remove bias* from a random bit sequence. It works by reading the bits in **pairs** of two:
-  `00` or `11` $\rightarrow$ discard
-  `01` $\rightarrow$ output `0`
-  `10` $\rightarrow$ output `1`

$\Rightarrow$ more **uniformly random sequence**, even if the original source has a slight bias.


In [95]:
def von_neumann_extraction(bits):
    extracted_bits = []
    for i in range(0, len(bits)-1, 2):
        pair = bits[i], bits[i+1]
        if pair == (0, 1):
            extracted_bits.append(0)
        elif pair == (1, 0):
            extracted_bits.append(1)
    return np.array(extracted_bits, dtype=np.uint8)

vn_bits = von_neumann_extraction(bitstring)
print('Initial bitstring length: ', len(bitstring))
print('Extracted (Von Neumann) bitstring length: ', len(vn_bits))
percent_survived = (len(vn_bits) / len(bitstring)) * 100
print(f'Percentage of bits that survived: {percent_survived:.2f}%')

unique, counts_vn = np.unique(vn_bits, return_counts=True)
counts_dict = {int(k): int(v) for k, v in zip(unique, counts_vn)}
print()
print('Initial counts: ', counts)
print('Von Neumann counts: ', counts_dict)

shots_vn = len(vn_bits)
if shots_vn > 0:
    probs_vn = counts_vn / shots_vn
    h_min_vn = -np.log2(np.max(probs_vn))
    print()
    print('Initial Min Entropy: ', h_min)
    print('Min Entropy after Von Neumann extraction: ', h_min_vn)
else:
    print('No bits survived the Von Neumann extraction')



Initial bitstring length:  1000000
Extracted (Von Neumann) bitstring length:  249706
Percentage of bits that survived: 24.97%

Initial counts:  {'1': 500058, '0': 499942}
Von Neumann counts:  {0: 124681, 1: 125025}

Initial Min Entropy:  0.9998326570809586
Min Entropy after Von Neumann extraction:  0.9980138820846688


#### Von Neumann extraction:
- pros: nice & simple method 

- cons: discards a lotttt of bits (~75%) $\Rightarrow$ wastes data
     - computational cost (extra memory & CPU cycles)
    - time inefficiency 

### Hash-based Extraction 

Hash functions = math function / algorithm that maps input data of any size to a fixed-size string output
- cryptographic hash functions (SHA-256, SHA-3, Blake2, MD5) $\rightarrow$ less secure now


*Hash-based Randomness Extraction model* uses hash functions as extractors bc they:
- get a biased random input & they output a uniform(-ish?) bitstring 
- retain nearly all entropy from the input without discarding most bits (unlike Von Neumann)